In [1]:
import os
import sys
import numpy as np
import pandas as pd
import random

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from utilities.data_downloader import train_val_test_downloader, choose_one_column
from utilities.plots import plt, CO
from utilities.dtw import DTWdistanceMatrix

In [4]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

In [5]:
random.seed(42)
np.random.seed(42)

# Analysis Pipeline

In [6]:
def make_pipe(eps:float, metric:str):
    """
    Makes pipeline
    """
    return Pipeline(
        [
         ('scaler', RobustScaler()),
        ] * int(metric!='precomputed') +
        [
         ('DBSCAN', DBSCAN(eps=eps, metric=metric)),
        ]
    )

# DTW distance on raw dataset

In [7]:
train, val, test, labels = train_val_test_downloader('original')
train, val, test = tuple(
    map(lambda df: df['lgRate'],
    (train, val, test))
)

Datasets downloaded
 - train  : 810 entries
 - val    : 174 entries
 - test   : 174 entries
 - labels : 1158 entries


In [8]:
X = [np.array([el for el in entry if 
               np.issubdtype(type(el), np.number)], dtype=float)
     for df in (train, val, test) for entry in df.values]

In [ ]:
distance_matrix = DTWdistanceMatrix(X)

 47%|██████████████████▉                     | 550/1158 [00:58<02:04,  4.89it/s]

In [ ]:
(distance_matrix >= 0).all()

In [ ]:
n_samples = len(X)
pairwise_distances = np.array(
    [distance_matrix[i, j] for i in range(n_samples) for j in range(i)]
)

In [ ]:
plt.hist(np.log10(pairwise_distances), bins=32)
plt.show()

In [ ]:
epsilon = np.logspace(-1, 3, 1000)

In [ ]:
n_classes = np.array([], dtype=int)
n_noisy = np.array([], dtype=int)

for eps in epsilon:
    pipe = make_pipe(eps=eps, metric='precomputed')
    preds = pipe.fit_predict(distance_matrix)
    classes, counts = np.unique(preds, return_counts=True)
    n_classes = np.append(n_classes, len(classes)-1)
    n_noisy = np.append(n_noisy, counts[0])

In [ ]:
fig, axes = plt.subplots(ncols=2)

axes[0].scatter(epsilon, n_classes, marker='s', s=4, color='dimgray')
axes[0].set_xscale('log')
axes[0].set_title('n_classes vs eps')
axes[1].scatter(epsilon, n_noisy, marker='s', s=4, color='xkcd:green')
axes[1].set_xscale('log')
axes[1].set_title('noisy samples vs eps')

plt.show()